Using Matern spatial covariance components instead of RQIso or Squared Exponential.

In [1]:
using TimeSeries
using DataFrames
using GaussianProcesses
using GaussianProcesses: Mean, Kernel, evaluate, metric, IsotropicData, VecF64
using GaussianProcesses: Stationary, KernelData, MatF64
import GaussianProcesses: optimize!, get_optim_target, cov, grad_slice!
import GaussianProcesses: num_params, set_params!, get_params, update_mll!, update_mll_and_dmll!
import GaussianProcesses: get_param_names, cov!, addcov!, multcov!
import Proj4
using Optim
using Distances
;

In [2]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
# plt.rc("figure", figsize=(6,4))
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
;

# Data Import and Preprocessing

In [6]:
include("iowa.jl")
iowa = prepare_iowa_data(data_dir);

# Matérn spatio-temporal kernel

In [7]:
mk1 = fix(Periodic(log(0.5), log(√(3.0)), log(24.0)), :lp)
mk2 = Mat32Iso(log(2.0), log(√3.0)) # short term Matern
mk3 = SEIso(log(2.0), log(√3.0)) # short term SE
mk4 = Mat32Iso(log(1.0*24), log(√10.0))
mk5 = RQIso(log(10.0*24), log(√20.0), 3.0)
mk_nonoise=mk1+mk2+mk3+mk4+mk5

temporal_minx = [-1.5563, -0.0368977, 1.06012, -1.38956, -1.19061, -0.0300269, -1.03743, 1.86563, 0.9376, 3.21125, 1.41512, 6.0]
set_params!(mk_nonoise, temporal_minx[2:end])

ksp1 = SEIso(log(2*10^5), log(1.0))
ksp2 = SEIso(log(2*10^5), log(1.0))
ksp3 = SEIso(log(2*10^5), log(1.0))
ksp4 = SEIso(log(2*10^5), log(1.0))
ksp5 = SEIso(log(2*10^5), log(1.0))
k_means = SEIso(log(1.0), log(20.0))

k_spatiotemporal = Masked(mk1, [1]) * Masked(ksp1, [2,3]) +
                     Masked(mk2, [1]) * Masked(ksp2, [2,3]) +
                     Masked(mk3, [1]) * Masked(ksp3, [2,3]) +
                     Masked(mk4, [1]) * Masked(ksp4, [2,3]) +
                     Masked(mk5, [1]) * Masked(ksp5, [2,3]) +
                     fix(Masked(k_means, [2,3]))
logNoise=temporal_minx[1]

-1.5563

# Optimization

In [8]:
@time opt_out = optim_kernel(k_spatiotemporal, 0.0, iowa[:isdSubset], iowa[:hourly_data], :NLopt);

8454.017348 seconds (56.88 M allocations: 990.357 GiB, 2.36% gc time)


In [11]:
opt_out[:mll] # optimized marginal log-likelihood

-46107.67901020296

In [12]:
print(opt_out[:hyp]) # fitted hyperparameters

[-1.73966, -0.223185, 1.02585, 13.6014, -0.0342734, -1.26907, -1.1741, 9.20607, 0.0165053, 1.18303, -0.418253, 12.068, 0.619177, 0.695832, 0.36827, 10.7382, -0.56933, 2.21545, 1.397, 3.0, 13.0806, -0.018121]

In [17]:
print("mk₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √(mk1.kern.σ2*ksp1.σ2))
@printf("l: %5.3f\n", √mk1.kern.ℓ2)
@printf("p: %5.0f hours\n", mk1.kern.p)
print("> spatial decay:\n")
@printf("l: %5.3f km\n", √ksp1.ℓ2 / 1000)

print("\nmk₂: Matérn \n=================\n")
@printf("σ: %5.3f\n", √(mk2.σ2*ksp2.σ2))
@printf("l: %5.3f hours\n", mk2.ℓ)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f km\n", √ksp2.ℓ2 / 1000)

print("\nmk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √(mk3.σ2*ksp3.σ2))
@printf("l: %5.3f hours\n", √mk3.ℓ2)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f km\n", √ksp3.ℓ2 / 1000)

print("\nmk₄: Matérn \n=================\n")
@printf("σ: %5.3f\n", √(mk4.σ2*ksp4.σ2))
@printf("l: %5.3f hours\n", mk4.ℓ)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f km \n", √ksp4.ℓ2 / 1000)

print("\nmk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √(mk5.σ2*ksp5.σ2))
@printf("l: %5.3f days\n", √mk5.ℓ2 / 24)
@printf("α: %5.3f\n",  mk5.α)
print("> spatial decay:\n")
@printf("l: %5.3f km\n", √ksp5.ℓ2 / 1000)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(opt_out[:hyp][1]))

mk₁: Periodic 
σ: 2.695
l: 0.800
p:    24 hours
> spatial decay:
l: 807.255 km

mk₂: Matérn 
σ: 0.314
l: 0.281 hours
> spatial decay:
l: 9.957 km

mk₃: SEIso 
σ: 1.223
l: 3.264 hours
> spatial decay:
l: 174.208 km

mk₄: Matérn 
σ: 0.818
l: 2.005 hours
> spatial decay:
l: 46.083 km 

mk₅: RQIso 
σ: 3.970
l: 0.382 days
α: 20.086
> spatial decay:
l: 479.565 km

σy: 0.176
